In [1]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [2]:
def MAPE_regression(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [3]:
#NuralNet (Regression)
class RegressionNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_train):
        pred_x = self.layer1(X_train)
        
        return pred_x
    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))
                
   
        

In [4]:
#NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_train):
        pred_x = self.layer1(X_train)
        
        return pred_x
    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [40]:

def CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    #신경망 생성하기
    model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "CNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
      # calculate AUC of model
    fpr, tpr, thresholds = roc_curve(y_test, pred_y_test)
    auc = roc_auc_score(y_test, pred_y_test)
      
    return  {"auc_score": auc, "fpr":fpr, "tpr":tpr}
    

In [41]:
def RNet(X_train, y_train, X_test, y_test , hiddenlayer, node, Ouputnode, epochs):
    
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 신경망 생성하기
    model = RegressionNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
            
    #학습되 모델 저장
    torch.save(model, "RNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
    MSE = mean_squared_error(y_test, pred_y_test)  # MSE 값은 출력하지 않아도 됩니다. RMSE를 구하기 위한 절차입니다.
    
    # 출력 대상입니다. 성능지표로써 3가지값이 출력되게 됩니다.
    RMSE = np.sqrt(MSE)
    MAPE = MAPE_regression(y_test, pred_y_test)
    R_squared = r2_score(y_test, pred_y_test)
    return {'pred_y' : pred_y_test, 'RMSE' : RMSE, 'MAPE' : MAPE, "R_squared" : R_squared}

In [42]:
df1 = pd.read_csv("ab_te.csv")
df2 = pd.read_csv("RMS_bearing.csv")

In [43]:
x = df1.iloc[:,[0, 1, 2, 3]]
y = df1.iloc[:,[4]]
x1 = df2.iloc[:,[0,1,2]]
y1 = df2.iloc[:,[3]]


X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

X1_train,X1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=33)

In [38]:
auc , fpr, tpr= CNet(X_train, y_train, X_test,y_test, 2, 5, 1, 10)
#CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):

Sequential(
  (Linear1): Linear(in_features=4, out_features=5, bias=True)
  (relu1): ReLU()
  (Linear2): Linear(in_features=5, out_features=5, bias=True)
  (relu2): ReLU()
  (Linear3): Linear(in_features=5, out_features=5, bias=True)
  (relu3): ReLU()
  (Linear4): Linear(in_features=5, out_features=1, bias=True)
)
0epoch // loss =-0.0
torch.FloatTensor
tensor([[-0.2137],
        [-0.1985],
        [-0.2117],
        [-0.2139],
        [-0.1754],
        [-0.2121],
        [-0.2096],
        [-0.2032],
        [-0.2048],
        [-0.2004],
        [-0.1987],
        [-0.2110],
        [-0.2126],
        [-0.2042],
        [-0.2046],
        [-0.2086],
        [-0.2124],
        [-0.2146],
        [-0.2077],
        [-0.2126],
        [-0.1896],
        [-0.2095],
        [-0.2225],
        [-0.1378],
        [-0.2087],
        [-0.2100],
        [-0.1988],
        [-0.2013],
        [-0.1989],
        [-0.2099],
        [-0.2147],
        [-0.2155],
        [-0.2133],
        [-0.2132],

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [10]:
pred_y1_test = RNet(X1_train, y1_train, X1_test, y1_test, 2, 5, 1, 10)


0epoch // loss =1.0361895874666516e-05


In [11]:
print(pred_y1_test)

{'pred_y': array([[0.05252028],
       [0.05328456],
       [0.05420646],
       [0.06804861],
       [0.06640811],
       [0.05514751],
       [0.05348556],
       [0.05286469],
       [0.0684787 ],
       [0.05510148],
       [0.05313033],
       [0.05324189],
       [0.05278341],
       [0.05376489],
       [0.05340834],
       [0.05405645],
       [0.05359069],
       [0.05904232],
       [0.05377343],
       [0.05809842],
       [0.06306078],
       [0.05320136],
       [0.05350685],
       [0.05317663],
       [0.05497015],
       [0.05665414],
       [0.05341689],
       [0.05326344],
       [0.0532288 ],
       [0.05416496],
       [0.05806811],
       [0.05620381],
       [0.05371985],
       [0.06892097],
       [0.0283269 ],
       [0.0591922 ],
       [0.05317321],
       [0.05412509],
       [0.0594434 ],
       [0.05387323],
       [0.05331795],
       [0.06411456],
       [0.05317381],
       [0.05317973],
       [0.05322108],
       [0.05356781],
       [0.05890553],
  

In [12]:
RNET = torch.load("RNet.pt")
RNET.eval()

RegressionNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=3, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [13]:
CNET = torch.load("CNet.pt")
CNET.eval()

ClassificationNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=4, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [14]:
X_test = torch.Tensor(X_test.values)
pred_y_test = CNET.predict(X_test)
print(pred_y_test)

tensor([[-0.4484],
        [-0.4484],
        [-0.4484],
        [-0.4483],
        [-0.4484],
        [-0.4484],
        [-0.4484],
        [-0.4484],
        [-0.4481],
        [-0.4484],
        [-0.4484],
        [-0.4473],
        [-0.4484],
        [-0.4484],
        [-0.4484],
        [-0.4482],
        [-0.4480],
        [-0.4482],
        [-0.4484],
        [-0.4482],
        [-0.4484],
        [-0.4484],
        [-0.4459],
        [-0.4484],
        [-0.4477],
        [-0.4483],
        [-0.4484],
        [-0.4484],
        [-0.4481],
        [-0.4482],
        [-0.4484],
        [-0.4483],
        [-0.4484],
        [-0.4484],
        [-0.4483],
        [-0.4484],
        [-0.4484],
        [-0.4481],
        [-0.4484],
        [-0.4484],
        [-0.4477],
        [-0.4483],
        [-0.4484],
        [-0.4484],
        [-0.4473],
        [-0.4483],
        [-0.4483],
        [-0.4484],
        [-0.4478],
        [-0.4484],
        [-0.4483],
        [-0.4484],
        [-0.

In [15]:
X1_test = torch.Tensor(X1_test.values)
pred_y_test = RNET.predict(X1_test)
print(pred_y_test)

tensor([[0.0525],
        [0.0533],
        [0.0542],
        [0.0680],
        [0.0664],
        [0.0551],
        [0.0535],
        [0.0529],
        [0.0685],
        [0.0551],
        [0.0531],
        [0.0532],
        [0.0528],
        [0.0538],
        [0.0534],
        [0.0541],
        [0.0536],
        [0.0590],
        [0.0538],
        [0.0581],
        [0.0631],
        [0.0532],
        [0.0535],
        [0.0532],
        [0.0550],
        [0.0567],
        [0.0534],
        [0.0533],
        [0.0532],
        [0.0542],
        [0.0581],
        [0.0562],
        [0.0537],
        [0.0689],
        [0.0283],
        [0.0592],
        [0.0532],
        [0.0541],
        [0.0594],
        [0.0539],
        [0.0533],
        [0.0641],
        [0.0532],
        [0.0532],
        [0.0532],
        [0.0536],
        [0.0589],
        [0.0535],
        [0.0538],
        [0.0582],
        [0.0562],
        [0.0574],
        [0.0562],
        [0.0531],
        [0.0647],
        [0

In [16]:
X1_test = torch.Tensor(X1_test.values)

TypeError: new(): data must be a sequence (got builtin_function_or_method)